In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 처음 사용자 입력 시 음식명으로 검색은 0, 재료명으로 검색은 1로 선택하게 하게 함.
# 0(음식명) 입력하면 음식 카테고리 리스트를, 1(재료명) 입력하면 재료 카테고리 리스트를  보여주고 다시 선택하게 함.
# 우선 0 -> 음식 카테고리 선택 -> 최종 음식명을 선택하게 됨
# 최종 음식명을 선택하게 되면 같은 이름의 요리명을 가진 레시피를 조회수와 스크랩수가 높은 순으로 3개를 추천해주는 시스템.
# 1을 선택하면 -> 재료 카테고리 선택 -> 최종 주 재료명을 선택하게 됨
# 최종 재료명을 선택하게 되면 해당 재료가 들어있는 요리를 찾아 조회수와 스크랩수가 높은 순으로 3개를 추천해주는 시스템.
# 이후 좀더 구체적으로 발전 할 계획.

In [ ]:
# 재료컬럼을text로 label을 요리명으로 train 한뒤 재료명을 입력하면 재료가 속해있는 요리를 추천.

In [ ]:
# 레시피 데이터 불러오기
reci = pd.read_excel('./10000recipe Preprocessing_수정_3.xlsx')
reci.head(2)

,레시피제목,요리명,요리재료내용,재료,조리순서,조회수,추천수,스크랩수,요리종류별명,요리재료별명,재료2,재료2_Okt,Nouns_Okt,재료_불용어 제외
0,백전백승! 실패없는 김치볶음밥,김치볶음밥,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,"밥,다진양파,다진김치,다진햄,설탕,김칫국물,고추장,부순참깨,들기름",[조리순서]\n1.밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기\n2.양념장...,366065,1324,6914,밥/죽/떡,쌀,"밥, 다진양파, 다진김치, 다진햄, 설탕","[('밥', 'Noun'), (',', 'Punctuation'), ('다진', '...","['밥', '양파', '김치', '햄', '설탕']","밥,다진양파,다진김치,다진햄,설탕"
1,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,감자조림,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,"감자,물,간장,설탕,물엿,다진마늘,참깨,참기름",[조리순서]\n1.감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감...,478527,1102,8221,밑반찬,채소류,"감자, 간장, 설탕, 물엿, 다진마늘","[('감자', 'Noun'), (',', 'Punctuation'), ('간장', ...","['감자', '간장', '설탕', '물엿', '마늘']","감자,간장,설탕,물엿,다진마늘"


In [ ]:
reci_ingre_food = reci[['재료','요리명']]

In [ ]:
reci_ingre_food.head()

,재료,요리명
0,"밥,다진양파,다진김치,다진햄,설탕,김칫국물,고추장,부순참깨,들기름",김치볶음밥
1,"감자,물,간장,설탕,물엿,다진마늘,참깨,참기름",감자조림
2,"김치,돼지고기,양파,파,김치국물,간장,고춧가루,다진마늘,다시다",김치찌개
3,"돼지고기,양파,청양고추,대파,설탕,고추장,간장,고춧가루,다진마늘,굴소스,올리고당",제육볶음
4,"고등어,무,양파,대파,청양고추,설탕,다진마늘,다진생강,된장,진간장,들기름,고추가루",고등어조림


In [ ]:
reci_ingre_food = reci_ingre_food.rename(columns={'재료':'text', '요리명':'label'})

In [ ]:
# 레이블 인코딩
label_encoder = LabelEncoder()
reci_ingre_food['label'] = label_encoder.fit_transform(reci_ingre_food['label'])

In [ ]:
# 텍스트 토큰화 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reci_ingre_food['text'])
sequences = tokenizer.texts_to_sequences(reci_ingre_food['text'])
padded_sequences = pad_sequences(sequences, padding='post')

In [ ]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, reci_ingre_food['label'],test_size=0.2, random_state=42)

모델 구축 및 학습_CNN사용.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Embedding

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
#  1.데이터 균형 확인
reci_ingre_food['label'].value_counts()

In [ ]:
# 데이터 클래스 불균형으로 RandomOverSampler 사용.
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
num_classes = len(label_encoder.classes_)

In [ ]:
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint

In [ ]:
# 모델을 저장할 파일 경로와 이름 지정
checkpoint_path = "best_model.h5"

In [ ]:
# ModelCheckpoint 콜백 설정
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      monitor = 'val_accuracy', #모델의 성능을 푱가할 지표 선택(검증 정확도를 기준으로 선택)
                                      verbose =1, # 로그 출력
                                      save_best_only = True,
                                      mode = 'max')  # 모니터링 지표의 최대값을 기준으로 모델 선택

In [ ]:
# 모델 구성
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index) +1 , output_dim = 128, input_length= padded_sequences.shape[1]))
model.add(Conv1D(128,5, activation = 'relu')) # 필터수는 128, 커널 크기는 5
model.add(Dropout(0.7))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.7))  # 드롭아웃 비율을 0.7로 설정
model.add(Dense(num_classes, activation = 'softmax'))  # num_classes는 클래스의 수

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 학습
# ModelCheckpoint 콜백 적용
history = model.fit(X_resampled, y_resampled, epochs=70, validation_data= (X_test, y_test), callbacks=[checkpoint_callback])

Epoch 1/70
223/225 [============================>.] - ETA: 0s - loss: 4.6854 - accuracy: 0.0350
Epoch 1: val_accuracy improved from -inf to 0.03000, saving model to best_model.h5
225/225 [==============================] - 3s 12ms/step - loss: 4.6829 - accuracy: 0.0353 - val_loss: 4.6869 - val_accuracy: 0.0300
Epoch 2/70
225/225 [==============================] - ETA: 0s - loss: 3.7380 - accuracy: 0.1653
Epoch 2: val_accuracy improved from 0.03000 to 0.16000, saving model to best_model.h5
225/225 [==============================] - 7s 33ms/step - loss: 3.7380 - accuracy: 0.1653 - val_loss: 4.2153 - val_accuracy: 0.1600
Epoch 3/70
223/225 [============================>.] - ETA: 0s - loss: 2.7900 - accuracy: 0.2993
Epoch 3: val_accuracy improved from 0.16000 to 0.36667, saving model to best_model.h5
225/225 [==============================] - 3s 13ms/step - loss: 2.7887 - accuracy: 0.2994 - val_loss: 3.7376 - val_accuracy: 0.3667
Epoch 4/70
220/225 [============================>.] - ETA: 0s

In [ ]:
from keras.models import load_model

In [ ]:
saved_model = load_model('best_model.h5')

In [ ]:
# 모델 평가
loss, accuracy = saved_model.evaluate(X_test, y_test)
print(f'Test Accuracy : {accuracy}')

10/10 [==============================] - 0s 12ms/step - loss: 1.6404 - accuracy: 0.7233
Test Accuracy : 0.7233333587646484


In [ ]:
# 테스트 데이터에 대한 예측 수행
y_pred = saved_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

10/10 [==============================] - 0s 7ms/step


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# 정밀도, 재현율, F1 점수 계산
print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       0.50      1.00      0.67         1
           5       0.50      1.00      0.67         1
           6       0.60      0.60      0.60         5
           7       0.50      0.50      0.50         4
           8       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          11       1.00      1.00      1.00         1
          12       0.00      0.00      0.00         3
          13       0.25      0.50      0.33         2
          14       0.25      0.25      0.25         4
          16       1.00      1.00      1.00         1
          17       0.83      0.83      0.83         6
          19       0.50      0.50      0.50         2
          21       1.00      0.25      0.40         4
          22       0.62    

c:\Users\tjoeun\miniconda3\envs\secondnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tjoeun\miniconda3\envs\secondnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tjoeun\miniconda3\envs\secondnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# 정밀도, 재현율, F1 점수 계산 해석.
1. 클래스 불균형 문제_여러 클래스의 샘플수가 매우 적고, 일부 클래스에대해 정밀도와 재현율이 0 -> 해당 클래스에 대한 예측이 전혀 맞지 않았다는것을 의미
2. 일부 클래스에서 높은 성능 -> 몇몇 클래스는 높은 정밀도, 재현율, F1 점수확인.(이 클래스에 대해서는 잘 작동하고 있음을 의미)
3. 전체 성능 : 전제 정확도는 0.72 -> 테스트 데이터의 72%를 정확히 예측하고 있음을 의미.
              매크로 평균(macro avg)와 가중 평균(weighted avg) F1 점수가 각각 0.52와 0.70으로, 일부 클래스가 모델의 성능을 끌어내리고 있음.



=> 데이터 증강과 RandomOverSampler를 함께 사용하여 클래스 불균형 문제를 더 효과적으로 해결
 (데이터 증강은 새로운 데이터를 생성하여 데이터셋의 크기를 늘리고, 오버샘플링은 기존 데이터를 복제하여 소수 클래스의 샘플 수를 늘림 )
   

In [ ]:
import random
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk
import os
import pickle

In [ ]:
# NLTK WordNet 데이터 설치
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 데이터 증강 함수
def synonym_replacement(sentence, n):
    words = word_tokenize(sentence)
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = wordnet.synsets(random_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

In [ ]:
# 데이터 증강 수행
augmented_texts = [synonym_replacement(text, 2) for text in reci_ingre_food['text']]

In [ ]:
# 기존 텍스트 데이터와 증강된 텍스트 데이터를 결합
combined_texts = reci_ingre_food['text'].tolist() + augmented_texts
combined_labels = reci_ingre_food['label'].tolist() * 2

In [ ]:
# 텍스트 토큰화 및 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_texts)
sequences = tokenizer.texts_to_sequences(combined_texts)
padded_sequences = pad_sequences(sequences, padding='post')

In [ ]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, combined_labels, test_size=0.2, random_state=42)

In [ ]:
# RandomOverSampler 사용
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
#데이터 형식을 numpy 배열로 변환
X_resampled = np.array(X_resampled)
y_resampled = np.array(y_resampled)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
# 모델 구성
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=padded_sequences.shape[1]))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# 체크포인트 콜백 설정
checkpoint_path = "best_model.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
# 모델 학습
model.fit(X_resampled, y_resampled, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint_callback])

Epoch 1/30
524/524 [==============================] - ETA: 0s - loss: 3.1982 - accuracy: 0.2957
Epoch 1: val_accuracy improved from -inf to 0.70333, saving model to best_model.h5
524/524 [==============================] - 6s 11ms/step - loss: 3.1982 - accuracy: 0.2957 - val_loss: 2.2159 - val_accuracy: 0.7033
Epoch 2/30
523/524 [============================>.] - ETA: 0s - loss: 0.9226 - accuracy: 0.7403
Epoch 2: val_accuracy improved from 0.70333 to 0.88167, saving model to best_model.h5
524/524 [==============================] - 5s 10ms/step - loss: 0.9219 - accuracy: 0.7404 - val_loss: 0.9062 - val_accuracy: 0.8817
Epoch 3/30
519/524 [============================>.] - ETA: 0s - loss: 0.4788 - accuracy: 0.8571
Epoch 3: val_accuracy improved from 0.88167 to 0.91833, saving model to best_model.h5
524/524 [==============================] - 5s 10ms/step - loss: 0.4777 - accuracy: 0.8574 - val_loss: 0.5797 - val_accuracy: 0.9183
Epoch 4/30
519/524 [============================>.] - ETA: 0s

In [ ]:
# 저장된 모델 불러오기
best_model = load_model(checkpoint_path)


In [ ]:
# 테스트 데이터에 대한 예측 수행
y_pred = best_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

19/19 [==============================] - 0s 7ms/step


In [ ]:
# 정밀도, 재현율, F1 점수 계산
print(classification_report(y_test, y_pred_classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      0.50      0.67         4
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         9
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      0.75      0.86         8
          13       1.00      1.00      1.00         4
          14       0.64      1.00      0.78         7
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         2
          17       1.00    

c:\Users\tjoeun\miniconda3\envs\secondnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tjoeun\miniconda3\envs\secondnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tjoeun\miniconda3\envs\secondnv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


전체 성능 : 전제 정확도는 0.95 -> 테스트 데이터의 95%를 정확히 예측하고 있음을 의미.
              매크로 평균(macro avg)와 가중 평균(weighted avg) F1 점수가 각각 0.91와 0.94로 높아짐

음식명은 0, 재료는 1로 선택하여 레시피 3가지 추천 함수에 적용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 벡터라이저를 저장하는 코드 (한 번만 실행)
prepare_and_save_vectorizer(reci_ingre_food, 'tfidf_vectorizer.pkl')

In [ ]:
# 전체 함수

#치환함수 정의
def normalize_word(word):
    replacements = {
        "달걀" :"계란",
        "계란": "계란",
        "오뎅":"어묵",
        "어묵":"어묵",
        "무우":"무",
        "무":"무",
        "대파":"파",
        "파":"파",
        "씨레기":"시래기",
        "시래기":"시래기"
    }
    return replacements.get(word,word)


# 사용자의 입력을 받는 함수 정의
def get_user_choice():
    choice = input("쿠킹파트너 : 음식명 검색은 0, 재료명 검색은 1을 입력해 주세요~^^")
    return choice


# 음식 카테고리 1 선택 함수 정의
def show_categories(df):
    categories = df['요리종류별명'].unique()
    for idx, category in enumerate(categories, start=1):
        print(f"{idx}.{category}")
    return categories


# 음식 카테고리 1 내 요리명 선택 함수 정의
def show_recipes_by_category(df, category):
    filterd_df = df[df['요리종류별명'] == category]
    recipes = filterd_df['요리명'].unique()
    for idx, recipe in enumerate(recipes, start=1):
        print(f"{idx}.{recipe}")
    return recipes


# 요리명 선택 후 추천 요리 방법 함수 정의
def recommend_recipes(df, recipe_name):
    filtered_df = df[df['요리명'].str.contains(recipe_name)]
    sorted_df = filtered_df.sort_values(by=['조회수', '스크랩수'], ascending=[False, False])
    top_recipes = sorted_df.head(3)

    responses = []
    for i, match in enumerate(top_recipes.iterrows(), start=1):  # 최대 3개까지 출력
        response = f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)

    return '\n'.join(responses)




# TfidfVectorizer를 사용해서 텍스트를 벡터화 하는 함수 정의
def prepare_and_save_vectorizer(df, vectorizer_path):
    texts = df['text'].tolist()
    vectorizer = TfidfVectorizer()
    vectorizer.fit(texts)

    with open(vectorizer_path, 'wb') as f:
        pickle.dump(vectorizer, f)


def load_vectorizer(vectorizer_path):
    with open(vectorizer_path, 'rb') as f:
        vectorizer = pickle.load(f)

    return vectorizer


# 모델 로드 함수 정의
def load_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model



#재료명을 기반으로 요리 레시피 추천 함수 정의
def recommend_recipes_by_ingredients_ml(model, vectorizer, ingredients):

    #  입력된 재료를 텍스트로 변환
    text = ' '.join(ingredients)

    # 벡터화된 텍스트를 이용하여 모델로 예측
    X_text = vectorizer.transform([text])
    predicted_class =  best_model.predict(X_text)[0]

    #예측된 클래스를 사용하여 해당 레시피를 추천
    recommended_recipes = reci_ingre_food[reci_ingre_food['label'] == predicted_class]['text']



    responses =[]
    for i , match in enumerate(recommend_recipes.iterrows(), start=1): #최대 3개까지 출력
        response =  f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)

    return '\n'.join(responses)





# 메인 실행 함수
def main():
    model = load_model('path_to_model_checkpoint.pkl')
    vectorizer = load_vectorizer('tfidf_vectorizer.pkl')

    user_choice = get_user_choice()

    if user_choice == '0':
        categories = show_categories(reci)
        category_choice = input("쿠킹파트너 : 1~10까지의 숫자를 입력하거나 카테고리명을 입력하세요: ")

        if category_choice.isdigit():
            category_choice = int(category_choice)
            if 1 <= category_choice <= len(categories):
                selected_category = categories[category_choice -1]
            else:
                print("쿠킹파트너 : 다시 선택해주세요~")
                return
        else:
            if category_choice in categories:
                selected_category = category_choice
            else:
                print("쿠킹파트너: 다시 선택해주세요~")
                return

        recipes = show_recipes_by_category(reci, selected_category)
        recipe_choice = input( "쿠킹파트너: 숫자를 입력하거나 요리명을 입력하세요: ")

        if recipe_choice.isdigit():
            recipe_choice = int(recipe_choice)
            if 1 <= recipe_choice <= len(recipes):
                selected_recipe = recipes[recipe_choice -1]
            else:
                print("쿠킹 파트너: 다시 선택해주세요~")
                return

        else:
            if recipe_choice in recipes:
                selected_recipe =recipe_choice
            else:
                print("쿠킹 파트너: 다시 선택해주세요~")
                return


        print(recommend_recipes(reci, selected_recipe))


    elif user_choice == '1':
        ingredients = input("쿠킹파트너: 재료를 3가지 입력하세요 (쉼표로 구분해주세요^^): ").split(',')
        ingredients =[normalize_word(ingredient.strip()) for ingredient in ingredients]
        if len(ingredients) !=3:
            print("쿠킹 파트너 : 재료를 정확히 3가지 입력해주세요~")
            return


        recommended_recipes = recommend_recipes_by_ingredients_ml(model, vectorizer, ingredients)
        print(recommend_recipes)

    else:
        print("쿠킹 파너트 : 0과 1중에 입력해주세요")

if __name__ == "__main__":
    main()

쿠킹 파트너 : 재료를 정확히 3가지 입력해주세요~
